In [1]:
import pandas as pd
import regex as re

In [2]:
data = pd.read_csv('/datasets/log_week_end.csv', names=['rn', 'loguser', 'q'])

C:\Users\123123\AppData\Local\Temp\ipykernel_3752\787480926.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/datasets/log_week_end.csv', names=['rn', 'loguser', 'q'])


In [3]:
data.sample(5)

,rn,loguser,q
350287,894812,dev_1314,from tbl_254928
343003,26659,etl_528,from tbl_124365
520128,189303,etl_1157,"JOIN tbl_33595,FROM tbl_33623"
125763,799023,etl_1151,"into tbl_55460,from tbl_56769,join tbl_56769,j..."
51841,215464,etl_1151,"INTO tbl_17046,join tbl_17046,FROM tbl_17045"


Собираем в списки все команды

In [4]:
intos, froms, joins = [], [], []

In [5]:
for x in data.q.str.split(','):
    for g in x:
        if 'from' in g:
            froms.append(g)
        if 'join' in g:
            joins.append(g)
        if 'into' in g:
            intos.append(g)

Оставляем только название таблиц

In [6]:
intos_tbs = [re.search('tbl_\d*', x).group(0) for x in intos]
froms_tbs = [re.search('tbl_\d*', x).group(0) for x in froms]
joins_tbs = [re.search('tbl_\d*', x).group(0) for x in joins]

Делаем датафрейм с окличество into и from (from только от пользователей dev)

In [7]:
df1 = pd.DataFrame(data = froms_tbs, columns=['col']).groupby('col').agg({'col': 'count'})
df1.columns= ['count_froms']
df1.reset_index(inplace=True)

**Делаем дф с кол-ом join**

In [8]:
df3 = pd.DataFrame(data = joins_tbs, columns=['col']).groupby('col').agg({'col': 'count'})
df3.columns= ['count_joins']
df3.reset_index(inplace=True)

In [9]:
# объединим кол-во join и from
df1 = df1.merge(df3, on ='col', how='outer')
# заменим пропуски на 0
df1 = df1.fillna(0)
# посчитаем кол-во извлечений
df1['count_selects'] = df1['count_froms'] + df1['count_joins']
df1.head()

,col,count_froms,count_joins,count_selects
0,tbl_10000,24.0,6.0,30.0
1,tbl_10001,24.0,6.0,30.0
2,tbl_100026,6.0,0.0,6.0
3,tbl_100027,6.0,0.0,6.0
4,tbl_100028,6.0,0.0,6.0


**Делаем дф с кол-ом into**

In [10]:
df2 = pd.DataFrame(data = intos_tbs, columns=['col']).groupby('col').agg({'col': 'count'})
df2.columns= ['count_intos']
df2.reset_index(inplace=True)

In [11]:
# объединяем извлечения и записи
df = df1.merge(df2, on ='col', how='outer')
# заменим пропуски на 0
df = df.fillna(0)
# добавляем соотношение записей/извлечений
df['usefull'] = df.count_intos / df.count_selects
df = df.sort_values(by='usefull', ascending = False)
df.sample(10)

,col,count_froms,count_joins,count_selects,count_intos,usefull
14223,tbl_185730,2.0,0.0,2.0,0.0,0.0
37823,tbl_12808,0.0,0.0,0.0,1.0,inf
29168,tbl_76354,1.0,0.0,1.0,1.0,1.0
38241,tbl_200839,0.0,0.0,0.0,4.0,inf
33320,tbl_92829,15.0,10.0,25.0,0.0,0.0
26224,tbl_53475,1.0,0.0,1.0,1.0,1.0
17883,tbl_208254,8.0,2.0,10.0,0.0,0.0
37308,tbl_83039,0.0,8.0,8.0,0.0,0.0
7568,tbl_14419,4.0,1.0,5.0,2.0,0.4
32368,tbl_89114,20.0,5.0,25.0,0.0,0.0
